In [11]:
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

dfs = defaultdict(lambda: defaultdict(dict))

for n_folds in [3]:
    for tuning_setting in ['fs', 'of']:
        for seed in [1,42]:
            try:
                dfs[n_folds][tuning_setting][seed] = pd.read_csv(f'data/dml_summary_VOCATIONAL_{n_folds}folds_{tuning_setting}_seed{seed}.csv')
                dfs[n_folds][tuning_setting][seed]['tuning'] = np.where(dfs[n_folds][tuning_setting][seed]['tuning']=='tune', tuning_setting, 'no_tune')
            except FileNotFoundError:
                print(f'File dml_summary_{n_folds}folds_{tuning_setting}_seed{seed}.csv not found, skipping.')

File dml_summary_5folds_fs_seed1.csv not found, skipping.
File dml_summary_5folds_fs_seed42.csv not found, skipping.
File dml_summary_5folds_of_seed42.csv not found, skipping.


In [13]:
all_df_list = []

for nfold in dfs:
    for setting in dfs[nfold]:
        for seed in dfs[nfold][setting]:
            dfs[nfold][setting][seed]['seed'] = seed
            dfs[nfold][setting][seed]['nfold'] = nfold
            all_df_list.append(dfs[nfold][setting][seed])

full_df = pd.concat(all_df_list, ignore_index=True)

In [14]:
full_df = full_df.drop_duplicates(subset=[col for col in full_df.columns if col != 'tuning_setting'])

In [15]:
full_df_42 = full_df[full_df['seed']==42]

In [16]:
full_df_42 = full_df_42[["tuning","model","coef_d","std err_d","nfold",'P>|t|_d']]

In [17]:
summary = full_df_42.pivot_table(
    index=['nfold', 'model'],
    columns='tuning',
    values=['coef_d', 'std err_d','P>|t|_d'],
)

summary.columns = [f'{stat}_{tune}' for stat, tune in summary.columns]
summary = summary.reset_index()


In [18]:
summary[summary['nfold']==3]

,nfold,model,P>|t|_d_fs,P>|t|_d_no_tune,P>|t|_d_of,coef_d_fs,coef_d_no_tune,coef_d_of,std err_d_fs,std err_d_no_tune,std err_d_of
0,3,lasso,0.237795,0.316510,0.393198,0.323099,-523.202013,0.297003,0.273693,522.337746,0.347846
1,3,rf,0.398913,0.112686,0.458926,0.306872,0.611963,0.271989,0.363780,0.385796,0.367246
2,3,xgb,0.347561,0.291162,0.571980,0.649155,83.876802,0.410161,0.691082,79.460705,0.725771
